In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [8]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.num_filters_total = num_filters * len(filter_sizes) # 3*3 = 9
        self.W = nn.Embedding(vocab_size, embedding_size) # 16,2 
        self.Weight = nn.Linear(self.num_filters_total, num_classes, bias=False) # 9,2
        self.Bias = nn.Parameter(torch.ones([num_classes])) # 2
        self.filter_list = nn.ModuleList([nn.Conv2d(1, num_filters, (size, embedding_size)) for size in filter_sizes]) # 1,3,2,2
        
    def forward(self, X):
        embedded_chars = self.W(X)   # [batch_size, sequence_length, sequence_length]
        embedded_chars = embedded_chars.unsqueeze(1)
        
        pooled_outputs = []
        for i, conv in enumerate(self.filter_list):
            h = F.relu(conv(embedded_chars))
            mp = nn.MaxPool2d((sequence_length - filter_sizes[i] + 1, 1))
            pooled = mp(h).permute(0, 3, 2, 1)
            pooled_outputs.append(pooled)
        
        h_pool = torch.cat(pooled_outputs, len(filter_sizes))
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total])
        model = self.Weight(h_pool_flat) + self.Bias
        return model        

In [2]:
embedding_size = 2
sequence_length = 3
num_classes = 2
filter_sizes = [2, 2, 2]
num_filters = 3
sentences = ['i love you', 'he loves me', 'she likes baseball', 'i hate you', 'sorry for that', 'this is awful']
labels = [1, 1, 1, 0, 0, 0]

In [3]:
word_list = ' '.join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

In [9]:
model = TextCNN()

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
inputs = torch.LongTensor([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences])
targets = torch.LongTensor([out for out in labels])

In [19]:
for epoch in range(5000):
    optimizer.zero_grad()
    output = model(inputs)  # [batch_size, num_classes]
    
    loss = criterion(output, targets)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch+1), 'cost=', '{:.6f}'.format(loss))
    
    loss.backward()
    optimizer.step()    

Epoch: 1000 cost= 0.001663
Epoch: 2000 cost= 0.000278
Epoch: 3000 cost= 0.000096
Epoch: 4000 cost= 0.000041
Epoch: 5000 cost= 0.000020


In [20]:
test_text = 'sorry hate she'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = torch.LongTensor(tests)

In [24]:
predict = model(test_batch).data.max(1, keepdim=True)[1]

In [26]:
if predict[0][0] == 0:
    print(test_text, 'is bad mean.')
else:
    print(test_text, 'is godd mean.')

sorry hate she is bad mean.
